In [ ]:
import gizmo_analysis as gizmo
import utilities as ut
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sl_utilities import distinct_colours as dc
from sl_utilities import distance_functions
import os
import pickle
from fof_analysis import fof

In [ ]:
simname = 'm12i_res7100_mhdcv'
simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_res7100/mhdcv/1Myr/fire2/'
snapnumber = 596

In [ ]:
part = gizmo.io.Read.read_snapshots(['all'],'snapshot_index', snapnumber, simulation_name=simname, simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)  
rxyz     = part['star'].prop('host.distance.total')
Rxy      = part['star'].prop('host.distance.principal.cylindrical')[:,0]
x        = part['star'].prop('host.distance.principal')[:,0]
y        = part['star'].prop('host.distance.principal')[:,1]
z        = part['star'].prop('host.distance.principal')[:,2] 
mass     = part['star'].prop('mass')
feh      = part['star'].prop('metallicity.fe')
mgh      = part['star'].prop('metallicity.mg')

ids      = part['star'].prop('id')
id_child = part['star'].prop('id.child')
age      = part['star'].prop('age')
#select young stars within the disk for star clusters
keep = np.where((age <= .003) & ((Rxy < 20) & (Rxy>2)) & (abs(z) < 1.5))
#to run cluster finding on
rxyz0     = rxyz[keep]
Rxy0      = Rxy[keep]
x0        = x[keep]
y0        = y[keep]
z0        = z[keep]
mass0     = mass[keep]
feh0      = feh[keep]


id0       = ids[keep]
id_child0 = id_child[keep]
age0      = age[keep]


In [10]:
linking_length = 0.01 #10 parsec (unit here is in kpc)
ncut           = 10 #5 star particles
ind, xcm, ycm, zcm, mtot, grpid, r90, r50, rmax =fof.find(x0,y0,z0, b=linking_length, mass=mass0, ncut=ncut)
ngroup = len(mtot)

linking length  0.01
min number particles in group  10
0.10273504257202148 seconds
number of groups 16
max number in a group 43


In [12]:
print(ind)

[[846, 847, 848, 849, 850, 853, 856, 859, 860, 862, 863, 864, 867, 868, 870, 871, 873, 874, 876, 877, 880, 881, 898, 869, 872, 852, 885, 858, 866, 865, 899, 875, 878, 882, 851, 883, 884, 854, 855, 857, 861, 886, 879], [908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 922, 923, 944, 929, 955, 928, 934, 942, 943, 945, 948, 924, 925, 941, 946, 920, 931, 926, 932, 947, 933, 940, 938, 935, 937, 936, 939, 949], [1064, 1068, 1070, 1095, 1091, 1084, 1071, 1074, 1088, 1089, 1090, 1094, 1069, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1085, 1086, 1073, 1087, 1066, 1067, 1092, 1093], [42, 356, 359, 362, 364, 367, 360, 365, 361, 358, 363, 1171, 1164, 1165, 1166, 1168, 1169, 1170], [785, 788, 789, 790, 791, 792, 794, 795, 807, 808, 809, 786, 803, 796, 793, 804, 810], [307, 308, 310, 311, 326, 309, 312, 313, 314, 322, 323, 324, 325, 318, 320, 327], [110, 114, 116, 117, 118, 120, 121, 122, 119, 123, 113, 125, 127, 111, 126], [1193, 1194, 1196, 1197, 1201, 1210, 1222, 1198, 1211,

In [16]:
export_cluster={}
for grp_index in range(ngroup):  #iterate over each group
    cluster={}
    ids_in_cluster = id0[ind[grp_index]]  #these are the star particle ids in each cluster
    id_children_in_cluster = id_child0[ind[grp_index]]
    age=age0[ind[grp_index]]
    nstar = len(ids_in_cluster)
    groupid=grpid[grp_index]
    print('------------------------------------------------------------------------------------------------------------------')
    print('grpid, nstar, xcm (kpc), ycm (kpc), zcm (kpc), mtot (msun), rmax (pc)')
    print('%s     %i     %.4f     %.4f    %.4f     %.2e     %.1f ' % (grpid[grp_index], nstar, xcm[grp_index], ycm[grp_index], zcm[grp_index], mtot[grp_index], 1000*rmax[grp_index]))
    print("age of stars in the cluster",age)
    print('ids')
    string = '[' 
    for i in ids_in_cluster:
        string = string + str(i) + ', '

    #get rid of last extra ,
    length = len(string)-2
    string = string[0:length] + ']'
    print(string)

    print('id children')
    string = '[' 
    for i in id_children_in_cluster:
        string = string + str(i) + ', '

    #get rid of last extra ,
    length = len(string)-2
    string = string[0:length] + ']'
    print(string)
    print("These are the ids printed",ids_in_cluster)
    #feh_in_cluster=feh0[ind[grp_index]]
    #cluster={"cluster_groupid":groupid,"no_of_star":nstar,"id":ids_in_cluster,"id_children":id_children_in_cluster,"xcm":xcm[grp_index],"ycm":ycm[grp_index],"zcm":zcm[grp_index],"mtot":mtot[grp_index],"r90":r90[grp_index],"r50":r50[grp_index],"rmax":rmax[grp_index],"x":x0[ind[grp_index]],"y":y0[ind[grp_index]],"z":z0[ind[grp_index]],"age":age0[ind[grp_index]],"feh":feh0[ind[grp_index]],"mgh":mgh0[ind[grp_index]],"ch":ch0[ind[grp_index]],"nh":nh0[ind[grp_index]],"oh":oh0[ind[grp_index]],"neh":neh0[ind[grp_index]],"sih":sih0[ind[grp_index]],"sh":sh0[ind[grp_index]],"cah":cah0[ind[grp_index]],"mgfe":mgfe0[ind[grp_index]],"ofe":ofe0[ind[grp_index]],"sife":sife0[ind[grp_index]],"sfe":sfe0[ind[grp_index]],"cafe":cafe0[ind[grp_index]],"nefe":nefe0[ind[grp_index]]}
    cluster={"cluster_groupid":groupid,"no_of_star":nstar,"id":ids_in_cluster,"id_children":id_children_in_cluster,"xcm":xcm[grp_index],"ycm":ycm[grp_index],"zcm":zcm[grp_index],"mtot":mtot[grp_index],"r90":r90[grp_index],"r50":r50[grp_index],"rmax":rmax[grp_index],"x":x0[ind[grp_index]],"y":y0[ind[grp_index]],"z":z0[ind[grp_index]],"age":age0[ind[grp_index]],"feh":feh0[ind[grp_index]]}

    export_cluster.update({groupid:cluster})


------------------------------------------------------------------------------------------------------------------
grpid, nstar, xcm (kpc), ycm (kpc), zcm (kpc), mtot (msun), rmax (pc)
1     43     -6.0073     6.0728    0.1123     4.95e+05     14.0 
age os stars in the cluster [2.24651479e-04 8.28341143e-04 1.34115589e-03 3.98085611e-04
 9.12414719e-05 1.52109695e-04 1.57946382e-04 8.28341143e-04
 1.05431180e-03 2.24651479e-04 2.13811888e-04 2.34657260e-04
 3.98085611e-04 3.78073916e-04 9.10891061e-04 6.84921765e-04
 1.40202732e-03 5.41503254e-04 1.95571435e-03 7.46625344e-04
 1.11252925e-04 8.04019418e-05 1.52109695e-04 1.13602866e-03
 5.21491438e-04 1.38201478e-03 8.69440557e-06 1.72955012e-04
 5.62348914e-04 3.88079761e-04 1.50459187e-03 1.62800345e-03
 2.54668835e-04 1.01247196e-04 6.03905145e-05 8.04019418e-05
 1.95339030e-05 1.22092469e-04 1.62949258e-04 1.72955012e-04
 2.34657260e-04 2.95395972e-05 1.25860448e-03]
ids
[21964074, 13285005, 30978536, 68630449, 45857179, 47786639, 

In [23]:
print(export_cluster[1]["x"])

[-6.00307966 -6.00349799 -5.99988107 -6.00310962 -6.0017005  -6.00654862
 -6.00698674 -6.00554434 -6.0050974  -6.00808298 -6.00762546 -6.00812528
 -6.01068292 -6.00328777 -6.00619153 -6.00745984 -6.00777392 -6.00678715
 -6.0006055  -6.00562482 -6.00646929 -6.00540532 -6.00503285 -6.00668641
 -6.00979852 -6.00419566 -6.00853963 -6.00964962 -6.0024964  -6.00964411
 -6.01415946 -6.0062386  -6.00951576 -6.00939451 -6.01092185 -6.01307867
 -6.00950509 -6.00960962 -6.00954438 -6.00964288 -6.01100886 -6.01159239
 -6.01534861]
